In [6]:
# Importar bibliotecas utilizadas
import pandas as pd

import numpy as np

import seaborn as sns

from sklearn.feature_extraction.text import CountVectorizer

# Regular expression
import re

from wordcloud import WordCloud

import matplotlib.pyplot as plt


In [7]:
# Set of predefined words NLTK (Natural Language Toolkit)
import nltk 

nltk.download('all')
set(stopwords.words('english'))

from nltk.tokenize import word_tokenize 

from nltk.corpus import stopwords 
#from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer

print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to
[nltk_data]    |     C:\Users\jcaet\AppData\Roaming\nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to
[nltk_data]    |     C:\Users\jcaet\AppData\Roaming\nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     C:\Users\jcaet\AppData\Roaming\nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     C:\Users\jcaet\AppData\Roaming\nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_ru is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     C:\Users\jcaet\AppData\Roaming\nltk_data...
[nltk_data]    | 

In [8]:
# Coleta dados
catalogoVulnerabilidades = pd.read_csv('dados/known_exploited_vulnerabilities.csv', header=0, sep=',')

# Criar coluna booleana state com 0 para Unknown e 1 para Known
catalogoVulnerabilidades['codigoKnownRansomwareCampaignUse'] = np.where(catalogoVulnerabilidades['knownRansomwareCampaignUse'] == 'Known', 1, 0 )

#Reordenando as colunas
catalogoVulnerabilidades = catalogoVulnerabilidades[['cveID', 
                                                     'vendorProject',
                                                     'product',
                                                     'vulnerabilityName',
                                                     'dateAdded',
                                                     'shortDescription',
                                                     'requiredAction',
                                                     'dueDate',
                                                     'knownRansomwareCampaignUse',
                                                     'codigoKnownRansomwareCampaignUse',
                                                     'notes']]    



In [9]:
# Verificar tipos de colunas
catalogoVulnerabilidades.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1083 entries, 0 to 1082
Data columns (total 11 columns):
 #   Column                            Non-Null Count  Dtype 
---  ------                            --------------  ----- 
 0   cveID                             1083 non-null   object
 1   vendorProject                     1083 non-null   object
 2   product                           1083 non-null   object
 3   vulnerabilityName                 1083 non-null   object
 4   dateAdded                         1083 non-null   object
 5   shortDescription                  1083 non-null   object
 6   requiredAction                    1083 non-null   object
 7   dueDate                           1083 non-null   object
 8   knownRansomwareCampaignUse        1083 non-null   object
 9   codigoKnownRansomwareCampaignUse  1083 non-null   int32 
 10  notes                             316 non-null    object
dtypes: int32(1), object(10)
memory usage: 89.0+ KB


In [10]:
# Excluir a coluna notes
catalogoVulnerabilidades.drop(columns=['notes'], inplace = True)

In [11]:
# Verificar alguns dados
catalogoVulnerabilidades.head()

,cveID,vendorProject,product,vulnerabilityName,dateAdded,shortDescription,requiredAction,dueDate,knownRansomwareCampaignUse,codigoKnownRansomwareCampaignUse
0,CVE-2021-27104,Accellion,FTA,Accellion FTA OS Command Injection Vulnerability,2021-11-03,Accellion FTA contains an OS command injection...,Apply updates per vendor instructions.,2021-11-17,Known,1
1,CVE-2021-27102,Accellion,FTA,Accellion FTA OS Command Injection Vulnerability,2021-11-03,Accellion FTA contains an OS command injection...,Apply updates per vendor instructions.,2021-11-17,Known,1
2,CVE-2021-27101,Accellion,FTA,Accellion FTA SQL Injection Vulnerability,2021-11-03,Accellion FTA contains a SQL injection vulnera...,Apply updates per vendor instructions.,2021-11-17,Known,1
3,CVE-2021-27103,Accellion,FTA,Accellion FTA Server-Side Request Forgery (SSR...,2021-11-03,Accellion FTA contains a server-side request f...,Apply updates per vendor instructions.,2021-11-17,Known,1
4,CVE-2021-21017,Adobe,Acrobat and Reader,Adobe Acrobat and Reader Heap-based Buffer Ove...,2021-11-03,Acrobat Acrobat and Reader contain a heap-base...,Apply updates per vendor instructions.,2021-11-17,Unknown,0


In [12]:
# Transformar tipos de dados
catalogoVulnerabilidades.dateAdded = pd.to_datetime(catalogoVulnerabilidades.dateAdded)
catalogoVulnerabilidades.dueDate = pd.to_datetime(catalogoVulnerabilidades.dueDate)
##Tranforma a coluna codigoKnownRansomwareCampaignUse para boolean
catalogoVulnerabilidades['codigoKnownRansomwareCampaignUse'] = catalogoVulnerabilidades['codigoKnownRansomwareCampaignUse'].astype('bool')
##Transforma colunas em string
catalogoVulnerabilidades['shortDescription'] = catalogoVulnerabilidades['shortDescription'].astype('string')

In [13]:
# Verificar tipos de colunas após transformação
catalogoVulnerabilidades.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1083 entries, 0 to 1082
Data columns (total 10 columns):
 #   Column                            Non-Null Count  Dtype         
---  ------                            --------------  -----         
 0   cveID                             1083 non-null   object        
 1   vendorProject                     1083 non-null   object        
 2   product                           1083 non-null   object        
 3   vulnerabilityName                 1083 non-null   object        
 4   dateAdded                         1083 non-null   datetime64[ns]
 5   shortDescription                  1083 non-null   string        
 6   requiredAction                    1083 non-null   object        
 7   dueDate                           1083 non-null   datetime64[ns]
 8   knownRansomwareCampaignUse        1083 non-null   object        
 9   codigoKnownRansomwareCampaignUse  1083 non-null   bool          
dtypes: bool(1), datetime64[ns](2), object(6), string

In [14]:
# Antes de limpar coluna
catalogoVulnerabilidades.shortDescription

0       Accellion FTA contains an OS command injection...
1       Accellion FTA contains an OS command injection...
2       Accellion FTA contains a SQL injection vulnera...
3       Accellion FTA contains a server-side request f...
4       Acrobat Acrobat and Reader contain a heap-base...
                              ...                        
1078    Microsoft Windows SmartScreen contains a secur...
1079    Cisco Adaptive Security Appliance (ASA) and Fi...
1080    Microsoft Exchange Server contains an unspecif...
1081    ConnectWise ScreenConnect contains an authenti...
1082    Microsoft Streaming Service contains an untrus...
Name: shortDescription, Length: 1083, dtype: string

In [15]:
# Função para limpar os textos das colunas do dataframe
def limpa_coluna(corpus):
    return re.sub(r'[^a-zA-Z0-9\s]', '', corpus)

In [21]:
# Limpa a coluna 
pd_catalogoVulnerabilidades = pd.DataFrame(catalogoVulnerabilidades)
pd_catalogoVulnerabilidades['shortDescription'] = catalogoVulnerabilidades['shortDescription'].apply(lambda x: limpa_coluna(str(x)))
pd_catalogoVulnerabilidades.shortDescription

0       Accellion FTA contains an OS command injection...
1       Accellion FTA contains an OS command injection...
2       Accellion FTA contains a SQL injection vulnera...
3       Accellion FTA contains a serverside request fo...
4       Acrobat Acrobat and Reader contain a heapbased...
                              ...                        
1078    Microsoft Windows SmartScreen contains a secur...
1079    Cisco Adaptive Security Appliance ASA and Fire...
1080    Microsoft Exchange Server contains an unspecif...
1081    ConnectWise ScreenConnect contains an authenti...
1082    Microsoft Streaming Service contains an untrus...
Name: shortDescription, Length: 1083, dtype: object

In [22]:
# Função de Tokenização
def tokenizacao (corpus):
    tokens = word_tokenize(corpus)
    return tokens

In [23]:
# Copia todos os registros tokenizando-os
pd_catalogoVulnerabilidades_tk = pd_catalogoVulnerabilidades.copy()
pd_catalogoVulnerabilidades_tk['shortDescription'] = pd_catalogoVulnerabilidades_tk['shortDescription'].apply(lambda x: tokenizacao(str(x)))

pd_catalogoVulnerabilidades_tk.shortDescription.head()

0    [Accellion, FTA, contains, an, OS, command, in...
1    [Accellion, FTA, contains, an, OS, command, in...
2    [Accellion, FTA, contains, a, SQL, injection, ...
3    [Accellion, FTA, contains, a, serverside, requ...
4    [Acrobat, Acrobat, and, Reader, contain, a, he...
Name: shortDescription, dtype: object

In [24]:
# Realiza a tokenização palavra por palavra 
tokens_contagem = ' '.join([word for word in pd_catalogoVulnerabilidades.shortDescription])

tokens_contagem = tokenizacao(tokens_contagem)

tokens_frequencia = nltk.FreqDist(tokens_contagem)

df_frequencia_tk = pd.DataFrame({"token": list(tokens_frequencia.keys()),
                                 "frequencia": list(tokens_frequencia.values())})


df_frequencia_tk.nlargest(columns = "frequencia", n = 10)




,token,frequencia
14,to,1070
7,vulnerability,1006
10,a,929
3,an,716
46,the,675
25,in,577
38,that,556
70,allows,537
44,code,530
71,remote,527


In [ ]:
# Gráfico de Pareto antes de remover stop words
plt.figure(figsize=(10, 7))
x = sns.barplot(data = df_frequencia_tk.nlargest(columns = "frequencia", n = 10), x = "token", y = "frequencia", color = 'blue')
x.set(ylabel = "Quantidade", xlabel = "Tokens", title = 'Contendo stop words')
plt.show()


In [ ]:
# Função remover stop words
def remove_stop_words(word_list, stop_word_list):
    lista_sem_stop_words = [word for word in word_list 
                            if word.lower() not in stop_word_list]
    return lista_sem_stop_words

In [ ]:
# Carrega lista de stop words do inglês
sw_vulnerabilidade_ciberneticas = ["could", "vulnerability"] # Ver quais stop words adicionar
sw_en = list(set(stopwords.words('english')))
sw_en_plus = sw_en + sw_vulnerabilidade_ciberneticas
tokens_sem_stop_words = remove_stop_words(tokens_contagem, sw_en_plus)

In [ ]:
# Cria DataFrame sem stop words

tokens_sem_stop_words_frequencia = nltk.FreqDist(tokens_sem_stop_words)

df_sem_stop_words_frequencia = pd.DataFrame({"token": list(tokens_sem_stop_words_frequencia.keys()),
                                 "frequencia": list(tokens_sem_stop_words_frequencia.values())})


df_sem_stop_words_frequencia.nlargest(columns = "frequencia", n = 10)

In [ ]:
# Gráfico de Pareto depois de remover stop words
plt.figure(figsize=(10, 7))
x = sns.barplot(data = df_sem_stop_words_frequencia.nlargest(columns = "frequencia", n = 10), x = "token", y = "frequencia", color = 'blue')
x.set(ylabel = "Quantidade", xlabel = "Tokens", title = 'Sem stop words')
plt.show()


In [ ]:
# Função de Lematização
def lematizacao (df, nome_coluna):
    df[nome_coluna] = df[nome_coluna].map(lambda x: WordNetLemmatizer.lemmatize(x))
    return df

In [ ]:
# Copia os registros lematizando-os


In [ ]:
# Vetorizar coluna
vetorizar = CountVectorizer(lowercase = False, max_features=100)
bow = vetorizar.fit_transform(catalogoVulnerabilidades.shortDescription)

print(bow.shape)

In [ ]:
# Tranforma matriz esparsa em Dataframe
df_shortDescription_bow = pd.DataFrame.sparse.from_spmatrix(bow, columns=vetorizar.get_feature_names_out())

df_shortDescription_bow.head()

In [ ]:
# Wordcloud
shortDescription_all_words = ' '.join([word for word in pd_catalogoVulnerabilidades.shortDescription])

# Quantidade de palavras
print(len(shortDescription_all_words))

In [ ]:
# Gerar wordcloud

## https://amueller.github.io/word_cloud/generated/wordcloud.WordCloud.html

shortDescription_wc = WordCloud(width= 800, height= 500, max_font_size = 110, collocations=False).generate(shortDescription_all_words)

In [ ]:
# Plotar wordcloud
plt.figure(figsize=(10, 7))
plt.imshow(shortDescription_wc, interpolation='bilinear') #ver outras interpolações
plt.axis("off")
plt.show()